In [8]:
from __future__ import absolute_import, division, print_function

import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dense
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten,Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
from IPython import display

In [3]:
# hyperparameters
n_critic = 5
lr_d = 1e-4
lr_g = 1e-4
img_size = [64,64]
BUFFER_SIZE = 142
BATCH_SIZE = 16

In [4]:
def create_data():
    counter=0
    images = []
    labels=[]
    print('converting images...')
    for img in os.listdir('data2'):
        images.append(np.asarray(PIL.Image.open('data2/'+img).resize((64,64)).convert('RGB')))
        labels.append(img.split('.')[0])
        counter+=1
    images = np.asarray(images).astype('float32')
    display.clear_output(wait=True)
    print('done converting')
  #do something similar for the labels
    return images  

In [5]:
images=create_data()

done converting


In [6]:
images = (images - 127.5) / 127.5

In [28]:
def generator(train=False):
    _ = inputs = Input((64,64,3))
    _ = Dense(4*4*512, use_bias=False)(_)
    _ = BatchNormalization()(_, training=train)
    _ = LeakyReLU(alpha=0.3)(_)
    
    _ = Reshape(, 4, 4, 512))(_)
    _ = Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False)(_)
    _ = BatchNormalization()(_, training=train)
    _ = LeakyReLU(alpha=0.3)(_)
    
    _ = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(_)
    _ = BatchNormalization()(_, training=train)
    _ = LeakyReLU(alpha=0.3)(_)

    _ = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(_)
    _ = BatchNormalization()(_, training=train)
    _ = LeakyReLU(alpha=0.3)(_)
    
    _ = Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False)(_)
    output = Activation("tanh")(_)
    
    return Model(inputs = inputs , outputs = output)

In [30]:
generator(train=True).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
dense_8 (Dense)              (None, 64, 64, 8192)      24576     
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 64, 8192)      32768     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 64, 64, 8192)      0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 128, 128, 256)     52428800  
_________________________________________________________________
batch_normalization_11 (Batc (None, 128, 128, 256)     1024      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 128, 128, 256)     0         
__________